In [1]:
import torch
print(f"Is CUDA available? {torch.cuda.is_available()}")
print(f"Current device: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

Is CUDA available? True
Current device: NVIDIA GeForce RTX 5060


**ResNet50**

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import os


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Data Pre-processing & Augmentation
# As per guidelines, we apply resizing and normalization
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

#data_dir = 'H:\TCV_PROJECT\dataset\Self_Collected_Data' 
data_dir = 'H:\TCV_PROJECT\dataset\Public_Data' 
dataset = datasets.ImageFolder(data_dir, transform=data_transforms)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_db, test_db = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = DataLoader(train_db, batch_size=32, shuffle=True)
test_loader = DataLoader(test_db, batch_size=32, shuffle=False)


model = models.resnet50(weights='IMAGENET1K_V1')
for param in model.parameters():
    param.requires_grad = False # Freeze layers for faster training on RTX 5060

# Adjust final layer for 4 classes
model.fc = nn.Linear(model.fc.in_features, 4)
model = model.to(device)

# Training
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)

for epoch in range(10):
    model.train()
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/10 completed.")

# Evaluation
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print(classification_report(y_true, y_pred, target_names=dataset.classes))

Using device: cuda
Epoch 1/10 completed.
Epoch 2/10 completed.
Epoch 3/10 completed.
Epoch 4/10 completed.
Epoch 5/10 completed.
Epoch 6/10 completed.
Epoch 7/10 completed.
Epoch 8/10 completed.
Epoch 9/10 completed.
Epoch 10/10 completed.
              precision    recall  f1-score   support

       Glass       0.97      0.88      0.92        83
       Metal       0.88      0.88      0.88        90
       Paper       0.82      0.82      0.82        34
     Plastic       0.81      0.89      0.85        87

    accuracy                           0.87       294
   macro avg       0.87      0.87      0.87       294
weighted avg       0.88      0.87      0.88       294



**HOG + SVM**

In [ ]:
import cv2
import numpy as np
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.metrics import classification_report

def extract_hog_features(data_loader):
    features, labels = [], []
    for inputs, lbls in data_loader:
        for img in inputs:
            # Convert to grayscale for HOG
            img_gray = cv2.cvtColor(img.permute(1, 2, 0).numpy(), cv2.COLOR_RGB2GRAY)
            fd = hog(img_gray, orientations=9, pixels_per_cell=(8, 8), 
                     cells_per_block=(2, 2))
            features.append(fd)
        labels.extend(lbls.numpy())
    return np.array(features), np.array(labels)

# Extract features from the loaders we built for ResNet
X_train_hog, y_train_hog = extract_hog_features(train_loader)
X_test_hog, y_test_hog = extract_hog_features(test_loader)

# Train SVM 
svm_model = SVC(kernel='linear', probability=True)
svm_model.fit(X_train_hog, y_train_hog)

# Predict and Report 
y_pred_hog = svm_model.predict(X_test_hog)
print("HOG + SVM Classification Report:")
print(classification_report(y_test_hog, y_pred_hog, target_names=dataset.classes))